## Hi, I made only one notebook for all assignament. Thanks for you review!!!

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.cm as cm
import json
from geopy.geocoders import Nominatim
import requests
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
import folium
print("Importados")

# <B> 1) Datas from Wikipedia page <B>

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
print("Hecho")

Hecho


In [3]:
post_codes = soup.find('table')
fields = post_codes.find_all('td')

post_codes_t = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    post_codes_t.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
dfpostC = pd.DataFrame(data=[post_codes_t, borough, neighbourhood]).transpose()
dfpostC.columns = ['Postcode', 'Borough', 'Neighbourhood']
dfpostC.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## This is the final output to first part

In [4]:
dfpostC['Borough'].replace('Not assigned', np.nan, inplace=True)
dfpostC.dropna(subset=['Borough'], inplace=True)

dfpostC.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
dfpostC = dfpostC.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
dfpostC.columns = ['Postcode', 'Borough', 'Neighbourhood']
dfpostC.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
dfpostC['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
df_pN = dfpostC
df_pN.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# <B> 2) Latitude and Longitude Datas <B>

In [7]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)
geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## The next output is the final dataframe to secont part

In [9]:
merged_data = pd.merge(dfpostC, geospatial_data, on='Postcode')
merged_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
merged_data['Coordinates'] = list(zip(merged_data['Latitude'], merged_data['Longitude']))
merged_data.head()
merged_data['Coordinates'] = merged_data['Coordinates'].apply(Point)

gdf = gpd.GeoDataFrame(merged_data, geometry='Coordinates')
merged_data
canada = merged_data
canada.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,POINT (43.80669 -79.19435)
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,POINT (43.78454 -79.16050)
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,POINT (43.76357 -79.18871)
3,M1G,Scarborough,Woburn,43.770992,-79.216917,POINT (43.77099 -79.21692)
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,POINT (43.77314 -79.23948)


# <B> 3) Cluster Map of Toronto <B>

In [57]:
address = 'Toronto, Ca'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Datos de direccion y mapa obtenidos")

Datos de direccion y mapa obtenidos


In [16]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

### no se pudo importar probar como from sklearn.metrics import jaccard_similarity_score

## The following cells build the cluster map

In [35]:
# Y = postcode x = Coordenadas  'Latitude'], canada['Longitude']
X = merged_data[[ 'Latitude','Longitude']]
canada_b = merged_data

In [51]:
segmetos = 6

clusters_ca = KMeans(n_clusters=segmetos, random_state=0).fit(X) 

label = clusters_ca.labels_[0:].tolist()
#'Cluster Labels'
canada_b['Cluster Labels'] = label
canada_b.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Coordinates,Cluster Labels
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,POINT (43.80669 -79.19435),4
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,POINT (43.78454 -79.16050),4
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,POINT (43.76357 -79.18871),4
3,M1G,Scarborough,Woburn,43.770992,-79.216917,POINT (43.77099 -79.21692),4
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,POINT (43.77314 -79.23948),4


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(segmetos)
ys = [i + x + (i*x)**2 for i in range(segmetos)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_b['Latitude'], canada_b['Longitude'], canada_b['Neighbourhood'], canada_b['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters